In [7]:
import numpy as np

def hes_transform(a):
    dim = a.shape[0]
    h = np.eye(dim)
    for i in range(0, dim-2):
        s = np.sign(a[i + 1, i]) * np.linalg.norm(a[i + 1:dim, i])
        mu = 1/np.sqrt(2 * s * (s - a[i + 1, i]))
        v = [0]
        for j in range(1, dim):
            if j < i+1:
                v.append(0)
            elif j == i+1:
                v.append(a[i + 1, i] - s)
            else:
                v.append(a[j, i])
        v = mu*np.matrix(v).transpose()
        h_ = (np.eye(dim) - 2*v*v.transpose())
        a = h_*a*h_
        h = h_*h
    return a, h

def givens_rotation(a):
    rows_, cols_ = np.shape(a)
    q = np.identity(rows_)
    r = np.copy(a)
    rows, cols = np.tril_indices(rows_, -1, cols_)
    for (row, col) in zip(rows, cols):
        if r[row, col] != 0:
            r1 = np.hypot(r[col, col], r[row, col])
            cos = r[col, col]/r1
            sin = -r[row, col]/r1
            g = np.identity(rows_)
            g[[col, row], [col, row]] = cos
            g[row, col] = sin
            g[col, row] = -sin
            r = np.dot(g, r)
            q = np.dot(q, g.T)
    return q, r

def qr_method(a, max_iter, diag_tolerance):
    #1
    dim = a.shape[0]
    a, h = hes_transform(a)
    lam = []

    for i in range(1, max_iter):
        #2
        q, r = givens_rotation(a - np.dot(np.eye(dim), a[-1, -1]))
        a = np.dot(r, q) + np.dot(np.eye(dim), a[-1, -1])

        #3
        if np.linalg.norm(a[-1, -2]) < diag_tolerance:
            lam.append(a[-1, -1])
            a = a[:-1, :-1]
            dim -= 1
            if dim == 1:
                lam.append(a[-1, -1])
                break

    return lam

In [8]:
n = np.random.randint(3, 5)
lambdas = np.random.rand(n)*n*100
Alpha = np.diag(lambdas)
C = np.matrix(np.random.randn(n, n))*n
A = np.linalg.inv(C)*Alpha*C

print("np.sort(lambdas) - np.sort(qr_method): ")
print(np.sort(lambdas) - np.sort(qr_method(A, 100, 0.1**8)))

np.sort(lambdas) - np.sort(qr_method): 
[-1.42108547e-14 -2.13162821e-14  0.00000000e+00  1.13686838e-13]
